In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import cv2
import matplotlib.pylab  as plt
import time
import os
#from libtiff import TIFF

In [2]:
tree = ET.parse('test.xml')
root = tree.getroot()

In [3]:
img = cv2.imread('test.jpg',cv2.IMREAD_COLOR)
rows,cols,ch = img.shape

## draw certain point

In [ ]:
for table in root.iter('table'):
    point = table[0].get('points').split()
    cell = []
    idx = 0
    for t in table.iter('cell'):
        if idx==3:   # point idx
            cell.append(t.find('Coords').get('points').split())
            break
        idx++

img_tmp = img
table_point = [(int(x[0]),int(x[1]))for x in [x.split(',') for x in point]]


cell_point = []
for c in cell:
    cell_point += c
cell_p = [(int(x[0]),int(x[1])) for x in [x.split(',') for x in cell_point]]

for p in table_point:
    cv2.circle(img_tmp,p, 4, (0,0,255), 4)
for c in cell_p:
    cv2.circle(img_tmp,c, 4, (0,0,255), 4)
    
height, width = img_tmp.shape[:2]
size = (int(width/4), int(height/4))  
img_shrink = cv2.resize(img_tmp, size, interpolation=cv2.INTER_AREA) 


cv2.imshow('image', img_shrink)
cv2.waitKey (10000) # 显示 10000 ms 即 10s 后消失

## draw points

In [37]:
img_tmp = img
table_point = [(int(x[0]),int(x[1]))for x in [x.split(',') for x in point]]


cell_point = []
for c in cell:
    cell_point += c
cell_p = [(int(x[0]),int(x[1])) for x in [x.split(',') for x in cell_point]]

for p in table_point:
    cv2.circle(img_tmp,p, 4, (0,0,255), 4)
for c in cell_p:
    cv2.circle(img_tmp,c, 4, (0,0,255), 4)
    
height, width = img_tmp.shape[:2]
size = (int(width/4), int(height/4))  
img_shrink = cv2.resize(img_tmp, size, interpolation=cv2.INTER_AREA) 


cv2.imshow('image', img_shrink)
cv2.waitKey (10000) # 显示 10000 ms 即 10s 后消失


255

## transform

In [59]:
xy = ([[int(x[0])/4,int(x[1])/4]for x in [x.split(',') for x in point]])

rows_t, cols_t = img_shrink.shape[:2]

pts1 = np.float32(xy)
pts2 = np.float32([[0,0],[rows_t,0],[rows_t, cols_t],[0,cols_t]])

M = cv2.getPerspectiveTransform(pts1,pts2)
dst = cv2.warpPerspective(img_shrink,M,(rows_t,cols_t))

cv2.imshow('image', dst)
cv2.waitKey (10000) # 显示 10000 ms 即 10s 后消失


255

## label

In [4]:
def str2point(point):
    rows = len(point)
    res = np.zeros((rows,2),dtype='int32')
    
    for idx in range(rows):
        res[idx,0], res[idx,1] = point[idx].split(',')
    return res

def isSorted(l,isAcs = True):
    if isAcs:
        return all(l[i] <= l[i+1] for i in range(len(l)-1))
    else:
        return all(l[i] >= l[i+1] for i in range(len(l)-1))
def isAlign(l):
    return all(abs(l[i]-l[i+1])<=30 for i in range(len(l)-1))

def extractPoint(table):
    table_point = table.find('Coords').get('points').split()
    table_point = str2point(table_point)
    
    cell_point = []
    cell_loc = []
    for t in table.iter('cell'):
        tmp = t.find('Coords').get('points').split()
        cell_point.append(str2point(tmp))
        
        sr = int(t.get('start-row'))
        er = int(t.get('end-row'))
        sc = int(t.get('start-col'))
        ec = int(t.get('end-col'))
        cell_loc.append(np.array([sr,er,sc,ec]))
    
    return table_point, cell_point, cell_loc

def checkPoint(point, row, col):
    N = point.shape[0]
    flag = True
    if  N == 2*(row+col):
        vline = point[0:row+1,:]
        flag1 = isSorted(vline[:,1]) and isAlign(vline[:,0])
        hline = point[row:col+row+1,:]
        flag2 = isSorted(hline[:,0]) and isAlign(hline[:,1])
        vline = point[col+row:col+2*row+1,:]
        flag3 = isSorted(vline[:,1], False) and isAlign(vline[:,0])
        hline = point[col+2*row:N,:]
        flag4 = isSorted(hline[:,0], False) and isAlign(hline[:,1])
        if not(flag1 and flag2 and flag3 and flag4):
            flag = False
    else:
        flag = False
    return flag

In [51]:
def preprocess(img_path, xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    rows,cols,ch = img.shape

    img_table = []
    fm_table = []
    
    for table in root.iter('table'):
        table_point, cell_point ,cell_loc = extractPoint(table)
        
        assert len(table_point)>=4, "table point less than 4"
        
        
        idx1 = np.argmin(np.array([x[1]+x[0] for x in table_point])) # top left
        idx2 = np.argmax(np.array([x[1]-x[0] for x in table_point])) # down left
        idx3 = np.argmax(np.array([x[1]+x[0] for x in table_point]))  # down right
        idx4 = np.argmin(np.array([x[1]-x[0] for x in table_point]))  # top right
        tmp = [table_point[idx1], table_point[idx2], table_point[idx3], table_point[idx4]]
        table_point = tmp
        
        table_row = table_point[1][1] - table_point[0][1]
        table_col = table_point[3][0] - table_point[0][0]

        #transformation
        pts1 = np.float32(table_point)
        pts2 = np.float32([[0, 0],[0, table_row],[table_col, table_row],[table_col, 0]])
        M = cv2.getPerspectiveTransform(pts1,pts2)

        #label
        horizon_map = np.zeros((rows,cols) ,dtype="uint8")
        vertical_map = np.zeros((rows,cols) ,dtype="uint8")

        assert len(cell_loc)==len(cell_point)
            
        
        for idx in range(len(cell_point)):
            
            cell_row = cell_loc[idx][1] - cell_loc[idx][0] + 1
            cell_col = cell_loc[idx][3] - cell_loc[idx][2] + 1
            N = cell_point[idx].shape[0]
            
            if checkPoint(cell_point[idx], cell_row, cell_col):          
                assert N==2*(cell_row+cell_col), cell_col               
                # horizontal line feature map
                for i in range(cell_row, cell_row+cell_col):
                    cv2.line(horizon_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][i+1]), 1, 20)           # visuable horizontal line
                for i in range(1, cell_row):
                    cv2.line(horizon_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][N-i-cell_col]), 2, 20)  # unvisuable horizontal line
                for i in range(cell_col+2*cell_row, N-1):
                    cv2.line(horizon_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][i+1]), 1, 20)           # visuable horizontal line
                cv2.line(horizon_map, tuple(cell_point[idx][N-1]), tuple(cell_point[idx][0]), 1, 20)

                # vertical line feature map
                for i in range(cell_row):
                    cv2.line(vertical_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][i+1]), 1, 20)           # visuable vertical line
                for i in range(cell_row+1, cell_col+cell_row):
                    cv2.line(vertical_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][N-i+cell_row]), 2, 20)  # unvisuable vertical line
                for i in range(cell_col+cell_row, cell_col+2*cell_row):
                    cv2.line(vertical_map, tuple(cell_point[idx][i]), tuple(cell_point[idx][i+1]), 1, 20)           # visuable vertical line
            else:
#                 print("%d\t%d\t%d\t%d" %(idx,N,cell_row,cell_col))
                idx1 = np.argmax(np.array([x[1]-x[0] for x in cell_point[idx]]))  # down left
                idx2 = np.argmax(np.array([x[1]+x[0] for x in cell_point[idx]]))  # down right
                idx3 = np.argmin(np.array([x[1]-x[0] for x in cell_point[idx]]))  # top right
                
#                 print(cell_point[idx])
#                 print("%d\t%d\t%d" %(idx1,idx2,idx3))
                
                if idx1>idx2:
                    cell=[]
                    cell.append(cell_point[idx][1:])
                    cell.append(cell_point[idx][0])
                    cell.reverse()
                    idx1 = N-idx1
                    idx2 = N-idx2
                    idx3 = N-idx3
                else:
                    cell = cell_point[idx]
                
                assert N==len(cell), "%d\t%d" %(N, len(cell))
                
                # horizontal line feature map
                for i in range(idx1, idx2):
                    cv2.line(horizon_map, tuple(cell[i]), tuple(cell[i+1]), 1, 20)           # visuable horizontal line
                for i in range(idx3, N-1):
                    cv2.line(horizon_map, tuple(cell[i]), tuple(cell[i+1]), 1, 20)           # visuable horizontal line
                cv2.line(horizon_map, tuple(cell[N-1]), tuple(cell[0]), 1, 20)

                # vertical line feature map
                for i in range(idx1):
                    cv2.line(vertical_map, tuple(cell[i]), tuple(cell[i+1]), 1, 20)           # visuable vertical line
                for i in range(idx2, idx3):
                    cv2.line(vertical_map, tuple(cell[i]), tuple(cell[i+1]), 1, 20)           # visuable vertical line

            
                flag1 = (idx1==cell_row)
                flag2 = ((idx2-idx1)==cell_col)
                flag3 = ((idx3-idx2)==cell_row)
                flag4 = ((N-idx3)==cell_col)
                
                 
                if flag1 and flag3:
                    for i in range(1, idx1):
                        cv2.line(horizon_map, tuple(cell[i]), tuple(cell[idx3-i]), 2, 20)  # unvisuable horizontal line
                else:
                    if flag1:
                        for i in range(1, idx1):
                            cv2.line(horizon_map, tuple(cell[i]), (cell[idx3][0],cell[i][1]), 2, 20)  # unvisuable horizontal line
                    elif flag3:
                        for i in range(idx2+1, idx3):
                            cv2.line(horizon_map, tuple(cell[i]), (cell[idx1][0],cell[i][1]), 2, 20)  # unvisuable horizontal line
                
                if flag2 and flag4:
                    for i in range(idx1+1, idx2):
                        cv2.line(vertical_map, tuple(cell[i]), tuple(cell[N-i+idx1]), 2, 20)  # unvisuable vertical line
                else:
                    if flag2:
                        for i in range(idx1+1, idx2):
                            cv2.line(vertical_map, tuple(cell[i]), (cell[i][0], cell[0][1]), 2, 20)  # unvisuable vertical line
                    elif flag4:
                        for i in range(idx3+1, N-1):
                            cv2.line(vertical_map, tuple(cell[i]), (cell[i][0], cell[idx1][1]), 2, 20)  # unvisuable vertical line
            
#                 cv2.line(vertical_map, tuple(cell_point[idx][0]), tuple(cell_point[idx][idx1]), 250, 20)
#                 cv2.line(horizon_map, tuple(cell_point[idx][idx1]), tuple(cell_point[idx][idx2]), 250, 20)
#                 cv2.line(vertical_map, tuple(cell_point[idx][idx2]), tuple(cell_point[idx][idx3]), 250, 20)
#                 cv2.line(horizon_map, tuple(cell_point[idx][idx3]), tuple(cell_point[idx][0]), 250, 20)

        img_tmp = cv2.warpPerspective(img,M,(table_col,table_row))
        img_table.append(img_tmp)
        
        horizon_map = cv2.warpPerspective(horizon_map,M,(table_col,table_row))
        vertical_map = cv2.warpPerspective(vertical_map,M,(table_col,table_row))
        fm_table.append((horizon_map,vertical_map))
        
        
    return img_table, fm_table

In [52]:
dir_path = r"D:\tablebank\ICDAR2019_cTDaR\ICDAR2019_cTDaR\training\TRACKB1\ground_truth"
save_path = r"D:\tablebank\ICDAR2019_cTDaR\ICDAR2019_cTDaR\training\TRACKB1\preprocess"
files =  os.listdir(dir_path)

image_name = []
for img_file in files:
    name = os.path.splitext(img_file)
    print("current file %s" %(name[0]))
    
    if name[1] == '.jpg' or name[1] == '.png':
        img_path = os.path.join(dir_path, img_file)
        xml_path = os.path.join(dir_path, name[0]+'.xml')
        img_table, fm_table = preprocess(img_path, xml_path)
        assert len(img_table)==len(fm_table), "image and feature map not match"
        
        for idx in range(len(img_table)):
            image_name.append(name[0]+ '_'+ str(idx))
            cv2.imwrite(os.path.join(save_path, name[0]+ '_'+ str(idx)+ '.jpg'), img_table[idx])
            cv2.imwrite(os.path.join(save_path, name[0]+ '_'+ str(idx)+ '_maskh.png'), fm_table[idx][0])
            cv2.imwrite(os.path.join(save_path, name[0]+ '_'+ str(idx)+ '_maskv.png'), fm_table[idx][1])
    if name[1] == '.tiff':
        img_path = os.path.join(dir_path, img_file)
        tif = TIFF.open(img_path, mode='r')
        for img in list(tif.iter_images()):
            tiff = img
        cv2.imwrite(os.path.join(dir_path, name[0]+'.jpg'))
        
        img_path = os.path.join(dir_path, name[0]+'.jpg')
        xml_path = os.path.join(dir_path, name[0]+'.xml')
        img_table, fm_table = preprocess(img_path, xml_path)
        assert len(img_table)==len(fm_table), "image and feature map not match"
        
        for idx in range(len(img_table)):
            image_name.append(name[0]+ '_'+ str(idx))
            cv2.imwrite(os.path.join(save_path, name[0]+ '_'+ str(idx)+ '.jpg'), img_table[idx])
            cv2.imwrite(os.path.join(save_path, name[0]+ '_'+ str(idx)+ '_maskh.png'), fm_table[idx][0])
            cv2.imwrite(os.path.join(save_path, name[0]+ '_'+ str(idx)+ '_maskv.png'), fm_table[idx][1])

current file cTDaR_t00000
current file cTDaR_t00000
current file cTDaR_t00001
current file cTDaR_t00001
current file cTDaR_t00002
current file cTDaR_t00002
current file cTDaR_t00003
current file cTDaR_t00003
current file cTDaR_t00004
current file cTDaR_t00004
current file cTDaR_t00005
current file cTDaR_t00005
current file cTDaR_t00006
current file cTDaR_t00006
current file cTDaR_t00007
current file cTDaR_t00007
current file cTDaR_t00008
current file cTDaR_t00008
current file cTDaR_t00009
current file cTDaR_t00009
current file cTDaR_t00010
current file cTDaR_t00010
current file cTDaR_t00018
current file cTDaR_t00018
current file cTDaR_t00019
current file cTDaR_t00019
current file cTDaR_t00020
current file cTDaR_t00020
current file cTDaR_t00021
current file cTDaR_t00021
current file cTDaR_t00022
current file cTDaR_t00022
current file cTDaR_t00023
current file cTDaR_t00023
current file cTDaR_t00024
current file cTDaR_t00024
current file cTDaR_t00025
current file cTDaR_t00025
current file

In [1]:
import torch

## split train and test

In [1]:
from utils import split_train_val

In [51]:
dir_path = r'/home/semisupervised/sanyou/table/preprocess'
save_path = r'/home/semisupervised/sanyou/table/code/data/cTDaR19_imagename.txt'
files =  os.listdir(dir_path)

image_name = []
with open(save_path,'a') as f:        
    for img_file in files:
        name = os.path.splitext(img_file)
        name.split
        f.write('%s\n'%(name[0]))

In [2]:
dir_path = r'/home/semisupervised/sanyou/table/preprocess'
filename_path = 'cTDaR19_imagename.txt'
split_train_val(dir_path, filename_path)

In [ ]:
a = 'asd_0_mask_h'
a.split

## predict

In [2]:
import argparse
import os

import numpy as np
import torch
import torch.nn.functional as F

from PIL import Image
from utils import dense_crf
from unet import UNet

import cv2

from torchvision import transforms
import pydensecrf.densecrf as dcrf

In [3]:
def mask_to_image(mask):
    return Image.fromarray((mask * 100).astype(np.uint8))

In [48]:
net = UNet(n_channels=3, n_classes=3)

net.cpu()
net.load_state_dict(torch.load('./ckpt/CP21.pth', map_location='cpu'))
print("Using CPU version of the net, this may be very slow")

imgpath = '/home/semisupervised/sanyou/table/preprocess/cTDaR_t00263_0.jpg'

img = Image.open(imgpath)
height, width = img.size[:2]
img_tmp = cv2.imread(imgpath)

net.eval()


size = (min(800, int(width/4)), min(500,int(height/4)))
normMean = [0.4948052, 0.48568845, 0.44682974]
normStd = [0.24580306, 0.24236229, 0.2603115]
normTransform = transforms.Normalize(normMean, normStd)
Transform = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    normTransform
])


img_tmp = cv2.resize(img_tmp, (size[1],size[0]))

img = Transform(img)
img = img.unsqueeze(0)

with torch.no_grad():
    vmasks_pred, hmasks_pred = net(img)

    
    mask_v = vmasks_pred.reshape(vmasks_pred.shape[0]*vmasks_pred.shape[1], vmasks_pred.shape[2]*vmasks_pred.shape[3])
    mask_h = hmasks_pred.reshape(hmasks_pred.shape[0]*hmasks_pred.shape[1], hmasks_pred.shape[2]*hmasks_pred.shape[3])
       
    mask_v = np.argmax(mask_v, 0)
    mask_h = np.argmax(mask_h, 0)
#     mask_v = mask_v[0,:]
#     mask_h = mask_h[0,:]
    
    mask_v = mask_v.reshape(vmasks_pred.shape[2], vmasks_pred.shape[3])
    mask_h = mask_h.reshape(hmasks_pred.shape[2], hmasks_pred.shape[3])
    
    
#     mask = np.dstack([(mask_h*100).numpy(), np.zeros(size), np.zeros(size)])
#     img_tmp = cv2.add(img_tmp, mask.astype(np.uint8))
#     cv2.imwrite('./data/test_img.jpg',img_tmp)


#    img = np.squeeze(img)
#     img = img.permute(1,2,0)
#     print(img.shape)
#     mask_h = dense_crf(np.array(img).astype(np.uint8), mask_h)
    
    mask_v = mask_v.numpy()
    result_v = mask_to_image(mask_v)
    result_v.save("./data/test_V.jpg")

    mask_h = mask_h.numpy()
    result_h = mask_to_image(mask_h)
    result_h.save("./data/test_H.jpg")

Using CPU version of the net, this may be very slow


In [50]:
img = Image.open('/home/semisupervised/sanyou/table/preprocess/cTDaR_t00040_0.jpg')
img.size

(1232, 1708)

In [5]:
os.getcwd()

'/home/semisupervised/sanyou/table/code'

In [24]:
a = np.array([
    [[[1,2],[1,2]],[[3,4],[3,4]],[[5,6],[5,6]]],
    [[[7,8],[7,8]],[[9,10],[9,10]],[[10,11],[10,11]]],
    [[[1,2],[1,2]],[[3,4],[3,4]],[[5,6],[5,6]]],
    [[[7,8],[7,8]],[[9,10],[9,10]],[[10,11],[10,11]]]])

In [26]:
a.reshape(a.shape[1], a.shape[0]*a.shape[2]*a.shape[3])

array([[ 1,  2,  1,  2,  3,  4,  3,  4,  5,  6,  5,  6,  7,  8,  7,  8],
       [ 9, 10,  9, 10, 10, 11, 10, 11,  1,  2,  1,  2,  3,  4,  3,  4],
       [ 5,  6,  5,  6,  7,  8,  7,  8,  9, 10,  9, 10, 10, 11, 10, 11]])

In [27]:
a.permute()

(4, 3, 2, 2)